In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
from belief.nli import load_nli_model, load_nli_tokenizer, pairwise_nli_precompute
from belief.evaluation import load_facts
from collections import defaultdict
import torch
import json

In [2]:
model = load_nli_model()
tokenizer = load_nli_tokenizer()

/home2/abhijit.manatkar/miniconda3/envs/advnlp/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
facts = load_facts('data/calibration_facts.json', num_batches=1)[0]
facts_by_entity = defaultdict(list)
for fact in facts:
    facts_by_entity[fact.subject].append(fact)

In [4]:
outs = []

chunk_size = 100

for entity in facts_by_entity:
    outs += pairwise_nli_precompute(facts_by_entity[entity], model, tokenizer, chunk_size)

21609it [00:42, 507.88it/s]                                                     
35721it [00:59, 597.13it/s]                                                     
18225it [00:32, 560.52it/s]                                                     
26569it [00:45, 579.37it/s]                                                     
20449it [00:35, 579.99it/s]                                                     
24336it [00:46, 528.69it/s]                                                     
19321it [00:35, 543.95it/s]                                                     


In [7]:
outs_dict = {}

for out in outs:
    src, dest, scores = out
    if src not in outs_dict:
        outs_dict[src] = {}
    outs_dict[src][dest] = scores

In [12]:
with open('cache/nli_constraints.json', 'w') as f:
    json.dump(outs_dict, f)